In [ ]:
import configparser
import psycopg2
import glob
import os
import pandas as pd  

# Путь к config
config_path = os.path.join(os.getcwd(), "config.ini")

# Чтение config
config = configparser.ConfigParser()
config.read(config_path, encoding='utf-8')

data_folder = config['Paths']['DATA_FOLDER']
DATABASE_CREDS = config['Database']

#Сбор файлов из папки  в  единый ДатаФрейм для  последующей  загрузки в БД

files = glob.glob(os.path.join(data_folder, '*.csv'))


def merge_csv_files(files_list):
    dfs = []
    
    for file in files_list:
        df = pd.read_csv(file, encoding='utf-8')
        dfs.append(df)

    merged_df = pd.concat(dfs, ignore_index=True)
    
    return merged_df

def upload_to_db(merged_df):

    
 conn=psycopg2.connect(
    host=DATABASE_CREDS['HOST'],
    database=DATABASE_CREDS['DATABASE'],
    user=DATABASE_CREDS['USER'],
    password=DATABASE_CREDS['PASSWORD'])
 
 cursor = conn.cursor() 

 

 for i , row in merged_df.iterrows():
    query = """
            INSERT INTO fake_sales (
                shop_num, 
                cash_num, 
                cheque_id, 
                date, 
                product, 
                category, 
                quantity, 
                price, 
                discount_perc
            ) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
    cursor.execute(query,(
                    row['shop_num'],
                    row['cash_num'],
                    row['cheque_id'],
                    row['date'],
                    row['product'],
                    row['category'],
                    row['quantity'],
                    row['price'],
                    row['discount_perc']))

 conn.commit()
 conn.close()

# Функция очистки папки после загрузки данных
def clean_folder(folder_path):
        
        for file in os.listdir(folder_path):
          os.remove(os.path.join(folder_path, file))

#Выполение 
def main():
   try:
      merged_df=merge_csv_files(files)
      upload_to_db(merged_df)
      #clean_folder(data_folder) Очистка папки поле загрузки данных в БД
      
   except Exception as e:
      print(f"Ошибка: {e}")
   

if __name__ == "__main__":
   main()